In [1]:
# Importamos todas las librerías necesarias
# ==============================================================================

import numpy as np#módulo para utilizar operaciones matemáticas
import pandas as pd#librería para manejar datos en formato dataframe
import matplotlib#librería de gráficos
from matplotlib import pyplot as plt#libería de gráficos
import seaborn as sns#librería de gráficos
sns.set()#establecemos el estilo de la librería seaborn
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing import image
from keras.applications import vgg16
import random

from pathlib import Path

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report,confusion_matrix
import cv2
import os

from typing import List, Tuple

%matplotlib inline 

tf.__version__

'2.3.0'

In [ ]:
tf.compat.v1.disable_eager_execution()
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)
with tf.compat.v1.Session() as sess:
    print (sess.run(c))

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices()))
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

Num GPUs Available:  4


In [3]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [4]:
sess = tf.compat.v1.Session()
print(sess.list_devices())

[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 3402572834833998012), _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, -8215864125092014234), _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 6921314656, 830105440810738269), _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, -1514059663956027839)]


In [5]:
tf.test.is_built_with_cuda()

True

In [6]:
# tf.test.is_gpu_available()
# WARNING:tensorflow:From C:\Users\jdani\AppData\Local\Temp\ipykernel_9344\337460670.py:1: is_gpu_available (from tensorflow.python.framework.test_util) is deprecated and will be removed in a future version.
# Instructions for updating:
# Use `tf.config.list_physical_devices('GPU')` instead.
# True
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
df = pd.read_csv(r'C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\full_df.csv')
df.head(10)

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg
3,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg
4,5,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",5_right.jpg
5,6,60,Male,6_left.jpg,6_right.jpg,macular epiretinal membrane,moderate non proliferative retinopathy，epireti...,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",6_right.jpg
6,7,60,Female,7_left.jpg,7_right.jpg,drusen,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",7_right.jpg
7,8,59,Male,8_left.jpg,8_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",8_right.jpg
8,9,54,Male,9_left.jpg,9_right.jpg,normal fundus,vitreous degeneration,0,0,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['O'],"[0, 0, 0, 0, 0, 0, 0, 1]",9_right.jpg
9,10,70,Male,10_left.jpg,10_right.jpg,epiretinal membrane,normal fundus,0,0,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",10_right.jpg


In [ ]:
df.shape

In [ ]:
df.isnull().sum()/len(df)

In [ ]:
import tensorflow as tf
from tensorflow import keras
tf.__version__

In [ ]:
keras.__version__

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full),(X_test, y_test) = fashion_mnist.load_data()

In [ ]:
len(X_train_full[1])

In [ ]:
y_train_full

In [ ]:
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000] , y_train_full[5000:]


In [ ]:
class_names = ["T-shit/top" , "Trouser" , "Pullover" , "Dress" , "Coat" , "Sandal" , "Shirt" , "Sneaker", "Bag", "Ankle boot"]

In [ ]:
class_names[y_train[0]] 

In [ ]:
y_valid[:10]

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(300, activation = "relu"))
model.add(keras.layers.Dense(100, activation = "relu"))
model.add(keras.layers.Dense(10, activation = "softmax"))

In [ ]:
model.summary()

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", 
             optimizer = "sgd",
             metrics = ["accuracy"])

In [ ]:
X_valid.shape

In [ ]:
y_train.shape

In [ ]:
print(X_valid.shape, X_train.shape, y_train.shape, y_valid.shape)

In [ ]:
history = model.fit(X_train, y_train, epochs = 30,
                   validation_data = (X_valid, y_valid))

In [ ]:
'''pd.DataFrame(history.history).plot(figsize = (8,5))
plt.grid(true)
plt.gca().set.ylim(0,1)
plt.show()'''

In [ ]:
pd.DataFrame(history.history).plot(figsize = (8,5))
plt.grid(True)
plt.gca()
plt.show()

In [ ]:
pd.DataFrame(history.history).plot(figsize = (8,5))

## Problema de regresión DL 

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
housing = fetch_california_housing()

In [ ]:
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform( X_test)

In [ ]:
model2 = keras.models.Sequential([
    keras.layers.Dense(30, activation = "relu", input_shape = X_train.shape[1:]),
    keras.layers.Dense(1)
])
model2.compile(loss = "mean_squared_error", optimizer = "sgd")

## Primer problema Deep Learning

In [ ]:
mnist_data = keras.datasets.mnist.load_data()

In [ ]:
X_train_full = mnist_data[0][0]
y_train_full = mnist_data[0][1]

X_test = mnist_data[1][0]
y_test = mnist_data[1][1]

##### O lo mismo que
((X_train_full, y_train_full), (X_test, y_test)) = mnist_data


X_train, X_valid = X_train_full[:55000]/255.0, X_train_full[55000:]/255.0
y_train, y_valid = y_train_full[:55000], y_train_full[55000:]

In [ ]:
len(X_valid)

In [ ]:
mnist_data[0][1]

In [ ]:
np.shape(X_train)

In [ ]:
3e-1

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(100, activation = "relu"))
model.add(keras.layers.Dense(100, activation = "relu"))
model.add(keras.layers.Dense(100, activation = "relu"))
model.add(keras.layers.Dense(10, activation = "softmax"))# 10 neurons as output layer. We have 10 possible outputs

model.compile(loss="sparse_categorical_crossentropy", 
             optimizer = keras.optimizers.SGD(learning_rate=0.3),
             metrics = ["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs = 30,
                   validation_data = (X_valid, y_valid),
                   callbacks=[keras.callbacks.EarlyStopping(patience = 10)])

## Problema TFM

Output reunión con tutores:
- Comentar casos particulares de pacientes que hemos encontrado como por ejemplo, que el target no es el mismo que el diagnóstico
- Explicar los datos y particularidades

In [ ]:
df = pd.read_excel(r'C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\ODIR-5K\ODIR-5K\data.xlsx').rename(columns=lambda x: x.replace("-", "_").replace(" ", "_").lower())
df

In [ ]:
test = pd.read_csv(r'C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\full_df.csv')
test.loc[test["ID"] == 0]


In [ ]:
df2 = pd.read_csv(r'C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\full_df.csv').rename(columns=lambda x: x.replace("-", "_").replace(" ", "_").lower())
df2 = df2.rename(columns = {'n': 'normal',
                        'd': 'diabetes',
                        'g': 'glaucoma',
                        'c': 'cataract',
                        'a': 'age_related_macular_degeneration',
                        'h': 'hypertension',
                        'm': 'pathological_myopia',
                        'o': 'other_diseases'})

df2 = pd.concat([df2, pd.get_dummies(df2["target"], prefix="target")], axis = 1)
df2 = df2.rename(columns = {'target_[1, 0, 0, 0, 0, 0, 0, 0]': 'target_normal',
                        'target_[0, 1, 0, 0, 0, 0, 0, 0]': 'target_diabetes',
                        'target_[0, 0, 1, 0, 0, 0, 0, 0]': 'target_glaucoma',
                        'target_[0, 0, 0, 1, 0, 0, 0, 0]': 'target_cataract',
                        'target_[0, 0, 0, 0, 1, 0, 0, 0]': 'target_age_related_macular_degeneration',
                        'target_[0, 0, 0, 0, 0, 1, 0, 0]': 'target_hypertension',
                        'target_[0, 0, 0, 0, 0, 0, 1, 0]': 'target_pathological_myopia',
                        'target_[0, 0, 0, 0, 0, 0, 0, 1]': 'target_other_diseases'})

df2.head()

In [ ]:
df2.iloc[0]

In [ ]:
#Checking if 
[print(df2.iloc[x]) for x in [0,3194]]


# Pasos a seguir
## 1. Ver cantidad de imágenes de ojos normales. Ver nulos, distribución de enfermedades, etc.
## 2. Ideas:
### 2.1. Comparar el resto de ojos con los normales y entrenar modelo para clasificar los problemas de ojos y comparar ML con DP
### 2.2. Tratar de hacer un clasficador multilabel con machine learning y comparar resultados con Deep Learning


In [ ]:
df2.info()

In [ ]:
df2.describe(include = "all")

Datos interesantes aquí: media de edad, edad mínima y máxima del paciente, el ID no nos vale de nada. Podemos ver a grandes rasgos cuales son las enfermedades más frecuentes fijándonos en las medias más altas (sin contar N, serían Diabetes y Otras enfermedades no catalogadas)

In [ ]:
sns.displot(df2, x="patient_age", element = "step", hue = "patient_sex")

In [ ]:
df_right_eye = df2.copy().iloc[:3194]

df_left_eye = df2.copy().iloc[3194:]
df_left_eye.head()


In [ ]:
df2["hypertension"].sum()

In [ ]:
df2['target_normal'].sum()

In [ ]:
def grouped_histogram_plots(df, cols = [], title = ""):
    
    df_ = df[cols].copy()
    # Create a figure with 8 subplots arranged in a 2x4 grid
    fig, ax = plt.subplots(figsize=(12, 6))

    # Loop over the columns of the DataFrame and plot a histogram in each subplot
    ax.hist(df_.values, bins=2, range=[-0.5, 1.5], label=cols, edgecolor='black')

    # Set the title, x-axis label and y-axis label
    ax.set_title(title)
    ax.set_xlabel('Values')
    ax.set_ylabel('Frequency')

    # Set the x-axis ticks and labels
    ax.set_xticks([0, 1])
    ax.set_xticklabels(['0', '1'])

    # Add a legend to the plot
    ax.legend()

    # Show the plot
    plt.show()

In [ ]:
df_left_eye.columns[7:15]

In [ ]:
grouped_histogram_plots(df_right_eye, cols = df_right_eye.columns[7:15], title="Enfermedades catalogadas en Ojo derecho")

In [ ]:
grouped_histogram_plots(df_left_eye, cols = df_left_eye.columns[7:15], title="Enfermedades catalogadas en Ojo Izquierdo")

We can frame the problem, as presented in the available dataset, in several different ways, e.g.:

* A multi-class multi-label classification problem, where each instance (image) can contain several classes (e.g., an image belonging to both Diabetes and Other classes).
* A multi-class classification problem, where each instance can belong to a single class (by using the final labels (target) as classes, e.g.).
* A binary classification problem, where only instances belonging to 2 classes are samples from the dataset (e.g., only Cataract and Normal instances).

### Problema abordado: Normal vs resto

In [10]:
# https://www.kaggle.com/code/ossingh6/cataract-detection-using-vgg19/notebook


###### APPROACH 5 ###### using vgg19
from tensorflow.keras.preprocessing.image import load_img,img_to_array
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.data import Dataset
from tensorflow.keras.applications.vgg19 import VGG19


print("--------Loading and preprocessing dataset--------")
# Load dataframe

df2 = pd.read_csv(r'C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\full_df.csv').rename(columns=lambda x: x.replace("-", "_").replace(" ", "_").lower())
df2 = df2.rename(columns = {'n': 'normal',
                        'd': 'diabetes',
                        'g': 'glaucoma',
                        'c': 'cataract',
                        'a': 'age_related_macular_degeneration',
                        'h': 'hypertension',
                        'm': 'pathological_myopia',
                        'o': 'other_diseases'})

df2 = pd.concat([df2, pd.get_dummies(df2["target"], prefix="target")], axis = 1)
df2 = df2.rename(columns = {'target_[1, 0, 0, 0, 0, 0, 0, 0]': 'target_normal',
                        'target_[0, 1, 0, 0, 0, 0, 0, 0]': 'target_diabetes',
                        'target_[0, 0, 1, 0, 0, 0, 0, 0]': 'target_glaucoma',
                        'target_[0, 0, 0, 1, 0, 0, 0, 0]': 'target_cataract',
                        'target_[0, 0, 0, 0, 1, 0, 0, 0]': 'target_age_related_macular_degeneration',
                        'target_[0, 0, 0, 0, 0, 1, 0, 0]': 'target_hypertension',
                        'target_[0, 0, 0, 0, 0, 0, 1, 0]': 'target_pathological_myopia',
                        'target_[0, 0, 0, 0, 0, 0, 0, 1]': 'target_other_diseases'})


######################################
# def is_normal(text):
#     if "normal" in text:
#         return 1
#     else:
#         return 0

# df2["left_normal"] = df2["left_diagnostic_keywords"].apply(lambda x: is_normal(x))
# df2["right_normal"] = df2["right_diagnostic_keywords"].apply(lambda x: is_normal(x))

# left_not_normal = df2.loc[(df2.normal ==0) & (df2.left_normal == 0)]["left_fundus"].reset_index(drop = True).sample(260,random_state=42).values
# right_not_normal = df2.loc[(df2.normal ==0) & (df2.left_normal == 0)]["right_fundus"].reset_index(drop = True).sample(260,random_state=42).values

# print("Number of images in left not normal: {}".format(len(left_not_normal)))
# print("Number of images in right not normal: {}".format(len(right_not_normal)))

# left_normal = df2.loc[(df2.normal == 1) & (df2.left_normal == 1)]["left_fundus"].sample(250,random_state=42).values
# right_normal = df2.loc[(df2.normal == 1) & (df2.right_normal == 1)]["right_fundus"].sample(250,random_state=42).values

# not_normal = np.concatenate((left_not_normal,right_not_normal),axis=0)
# normal = np.concatenate((left_normal,right_normal),axis=0)

######################################

######################################
def is_cataract(text):
    if "cataract" in text:
        return 1
    else:
        return 0

df2["left_cataract"] = df2["left_diagnostic_keywords"].apply(lambda x: is_cataract(x))
df2["right_cataract"] = df2["right_diagnostic_keywords"].apply(lambda x: is_cataract(x))

left_cataract = df2.loc[(df2.cataract ==1) & (df2.left_cataract == 1)]["left_fundus"].values
right_cataract = df2.loc[(df2.cataract ==1) & (df2.right_cataract == 1)]["right_fundus"].values

print("Number of images in left not normal: {}".format(len(left_cataract)))
print("Number of images in right not normal: {}".format(len(right_cataract)))

left_normal = df2.loc[(df2.cataract ==0) & (df2["left_diagnostic_keywords"] == "normal fundus")]["left_fundus"].sample(250,random_state=42).values
right_normal = df2.loc[(df2.cataract ==0) & (df2["right_diagnostic_keywords"] == "normal fundus")]["right_fundus"].sample(250,random_state=42).values


not_normal = np.concatenate((left_cataract,right_cataract),axis=0) ## cataract
normal = np.concatenate((left_normal,right_normal),axis=0)
######################################


print(len(not_normal),len(normal))

--------Loading and preprocessing dataset--------
Number of images in left not normal: 304
Number of images in right not normal: 290
594 500


In [11]:
df2.columns

Index(['id', 'patient_age', 'patient_sex', 'left_fundus', 'right_fundus',
       'left_diagnostic_keywords', 'right_diagnostic_keywords', 'normal',
       'diabetes', 'glaucoma', 'cataract', 'age_related_macular_degeneration',
       'hypertension', 'pathological_myopia', 'other_diseases', 'filepath',
       'labels', 'target', 'filename', 'target_other_diseases',
       'target_pathological_myopia', 'target_hypertension',
       'target_age_related_macular_degeneration', 'target_cataract',
       'target_glaucoma', 'target_diabetes', 'target_normal', 'left_cataract',
       'right_cataract'],
      dtype='object')

In [12]:
# df_normal_vs_rest = df2.copy()
# df_normal_vs_rest = df_normal_vs_rest[["filename", "target_normal"]]

# df_normal_vs_rest.head(2)
# normal = df_normal_vs_rest[df_normal_vs_rest.loc[:, "target_normal"] == 1]["filename"]
# not_normal = df_normal_vs_rest[df_normal_vs_rest.loc[:, "target_normal"] == 0]["filename"]

# print(len(normal))
# print(len(not_normal))

dataset_dir = r'C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images'
image_size=224
labels = []
dataset = []
def create_dataset(image_category,label):
    for img in image_category:
        image_path = os.path.join(dataset_dir,img)
        try:
            image = cv2.imread(image_path,cv2.IMREAD_COLOR)
            image = cv2.resize(image,(image_size,image_size))
            print(f"Loaded image: {image_path}")
        except:
            print(f"Error with image {image_path}")
            continue
        
        dataset.append([np.array(image),np.array(label)])
    random.shuffle(dataset)
    return dataset

dataset = create_dataset(not_normal,1)
dataset = create_dataset(normal,0)

x = np.array([i[0] for i in dataset]).reshape(-1,image_size,image_size,3)
y = np.array([i[1] for i in dataset])
print(len(x))
print(len(y))
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

# Define the model
vgg = VGG19(weights="imagenet",include_top = False,input_shape=(image_size,image_size,3))
for layer in vgg.layers:
    layer.trainable = False
model = tf.keras.Sequential()
model.add(vgg)
model.add(Flatten())
model.add(Dense(1,activation="sigmoid"))
model.summary()

# Compile the model
model.compile(optimizer="adam",
              loss="binary_crossentropy",metrics=["accuracy"])

# Define callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",patience=10,verbose=1)
checkpoint = tf.keras.callbacks.ModelCheckpoint("vgg19.h5",monitor="val_accuracy",verbose=1,save_best_only=True,
                             save_weights_only=False,save_freq="epoch")


# Train the model
history = model.fit(x_train,y_train, batch_size = 16, epochs=50, validation_data=(x_test,y_test), callbacks=[checkpoint , early_stopping ])


Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\0_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\81_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\103_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\119_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\254_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\294_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\330_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\448_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\465_left

Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2147_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2150_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2151_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2152_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2153_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2154_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2155_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2156_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_imag

Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\254_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\294_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\330_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\448_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\465_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\477_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\553_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\560_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\594_l

Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2150_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2151_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2152_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2153_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2154_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2155_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2156_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2157_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_imag

Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\477_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\553_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\560_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\625_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\726_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\769_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\949_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\955_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_imag

...

Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\740_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2421_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\3203_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2533_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\4117_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\3324_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\3172_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2442_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_image

Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\3065_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2845_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\227_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\434_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2590_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2689_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\3358_left.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2964_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_image

Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2914_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\951_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\3096_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\1039_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2925_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\3246_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\3293_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\689_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocesse

Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\3116_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2911_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\3234_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\73_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2953_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\4354_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2771_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\3328_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocesse

Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\1061_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\3256_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2991_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\4156_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\3057_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\2340_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\1075_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images\3131_right.jpg
Loaded image: C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preproces

In [36]:
loss,accuracy = model.evaluate(x_test,y_test)
print("loss:",loss)
print("Accuracy:",accuracy)

7/7 [==============================] - 0s 38ms/step - loss: 0.6942 - accuracy: 0.4862
loss: 0.6942427754402161
Accuracy: 0.4862385392189026


In [14]:
df_normal_vs_rest[df_normal_vs_rest.loc[:, "target_normal"] == 1]["filename"]

0         0_right.jpg
1         1_right.jpg
7         8_right.jpg
9        10_right.jpg
24       29_right.jpg
            ...      
6356    4637_left.jpg
6358    4639_left.jpg
6360    4641_left.jpg
6378    4672_left.jpg
6386    4683_left.jpg
Name: filename, Length: 2873, dtype: object

In [9]:
###### APPROACH 4 ###### using vgg19

import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.data import Dataset
from tensorflow.keras.applications.vgg19 import VGG19


print("--------Loading and preprocessing dataseet--------")
# Load dataframe

df2 = pd.read_csv(r'C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\full_df.csv').rename(columns=lambda x: x.replace("-", "_").replace(" ", "_").lower())
df2 = df2.rename(columns = {'n': 'normal',
                        'd': 'diabetes',
                        'g': 'glaucoma',
                        'c': 'cataract',
                        'a': 'age_related_macular_degeneration',
                        'h': 'hypertension',
                        'm': 'pathological_myopia',
                        'o': 'other_diseases'})

df2 = pd.concat([df2, pd.get_dummies(df2["target"], prefix="target")], axis = 1)
df2 = df2.rename(columns = {'target_[1, 0, 0, 0, 0, 0, 0, 0]': 'target_normal',
                        'target_[0, 1, 0, 0, 0, 0, 0, 0]': 'target_diabetes',
                        'target_[0, 0, 1, 0, 0, 0, 0, 0]': 'target_glaucoma',
                        'target_[0, 0, 0, 1, 0, 0, 0, 0]': 'target_cataract',
                        'target_[0, 0, 0, 0, 1, 0, 0, 0]': 'target_age_related_macular_degeneration',
                        'target_[0, 0, 0, 0, 0, 1, 0, 0]': 'target_hypertension',
                        'target_[0, 0, 0, 0, 0, 0, 1, 0]': 'target_pathological_myopia',
                        'target_[0, 0, 0, 0, 0, 0, 0, 1]': 'target_other_diseases'})


# Continue your dataframe processing...

df_normal_vs_rest = df2.copy()
df_normal_vs_rest = df_normal_vs_rest[["filename", "target_normal"]]

# Splitting the dataset
train_df, valid_df = train_test_split(df_normal_vs_rest, test_size=0.2, random_state=42)

data_dir = r'C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images'

# Load and preprocess the images
img_size = 224
def load_and_preprocess_images(df):

    dataset = []
    for index, row in df.iterrows():
        img = cv2.imread(data_dir + '/' + row["filename"], cv2.IMREAD_COLOR)
        img = cv2.resize(img, (img_size, img_size))
#         img = img / 255.0
        dataset.append([np.array(image),np.array(row["target_normal"])])
#         images.append(img)
#         labels.append(row["target_normal"])
        random.shuffle(dataset)
    return dataset

print("--------Loading and processing images--------")
train_ds = load_and_preprocess_images(train_df)
valid_ds = load_and_preprocess_images(valid_df)

train_ds = np.array([i[0] for i in train_images]).reshape(-1,img_size,img_size,3)
valid_ds = np.array([i[0] for i in valid_images]).reshape(-1,img_size,img_size,3)

# Convert the data to TensorFlow Dataset
# train_ds = Dataset.from_tensor_slices((train_images, train_labels))
# valid_ds = Dataset.from_tensor_slices((valid_images, valid_labels))

# Shuffle the training dataset
# train_ds = train_ds.shuffle(buffer_size=10000, reshuffle_each_iteration=True).batch(16)
# valid_ds = valid_ds.batch(16)

# Define the model
# vgg = VGG19(weights="imagenet",include_top = False,input_shape=(img_size,img_size,3))
# for layer in vgg.layers:
#     layer.trainable = False
# model = tf.keras.Sequential()
# model.add(vgg)
# model.add(Flatten())
# model.add(Dense(1,activation="sigmoid"))
# model.summary()

# # Compile the model
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#               loss="binary_crossentropy",metrics=["accuracy"])

# # Define callbacks
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_acc",patience=5,verbose=1)
# checkpoint = tf.keras.callbacks.ModelCheckpoint("vgg19.h5",monitor="val_acc",verbose=1,save_best_only=True,
#                              save_weights_only=False,period=1)


# # Train the model
# history = model.fit(train_ds,  epochs=50, validation_data=valid_ds, callbacks=[early_stopping, checkpoint])


--------Loading and preprocessing dataseet--------
--------Loading and processing images--------


ValueError: cannot reshape array of size 3435936 into shape (224,224,3)

In [5]:
###### APPROACH 1 ######

import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.data import Dataset

# Load dataframe

df2 = pd.read_csv(r'C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\full_df.csv').rename(columns=lambda x: x.replace("-", "_").replace(" ", "_").lower())
df2 = df2.rename(columns = {'n': 'normal',
                        'd': 'diabetes',
                        'g': 'glaucoma',
                        'c': 'cataract',
                        'a': 'age_related_macular_degeneration',
                        'h': 'hypertension',
                        'm': 'pathological_myopia',
                        'o': 'other_diseases'})

df2 = pd.concat([df2, pd.get_dummies(df2["target"], prefix="target")], axis = 1)
df2 = df2.rename(columns = {'target_[1, 0, 0, 0, 0, 0, 0, 0]': 'target_normal',
                        'target_[0, 1, 0, 0, 0, 0, 0, 0]': 'target_diabetes',
                        'target_[0, 0, 1, 0, 0, 0, 0, 0]': 'target_glaucoma',
                        'target_[0, 0, 0, 1, 0, 0, 0, 0]': 'target_cataract',
                        'target_[0, 0, 0, 0, 1, 0, 0, 0]': 'target_age_related_macular_degeneration',
                        'target_[0, 0, 0, 0, 0, 1, 0, 0]': 'target_hypertension',
                        'target_[0, 0, 0, 0, 0, 0, 1, 0]': 'target_pathological_myopia',
                        'target_[0, 0, 0, 0, 0, 0, 0, 1]': 'target_other_diseases'})


# Continue your dataframe processing...

df_normal_vs_rest = df2.copy()
df_normal_vs_rest = df_normal_vs_rest[["filename", "target_normal"]]

# Splitting the dataset
train_df, valid_df = train_test_split(df_normal_vs_rest, test_size=0.2, random_state=42)

data_dir = r'C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images'

# Load and preprocess the images
def load_and_preprocess_images(df):
    img_size = 224
    images = []
    labels = []
    for index, row in df.iterrows():
        img = cv2.imread(data_dir + '/' + row["filename"])
        img = cv2.resize(img, (img_size, img_size))
        img = img / 255.0
        images.append(img)
        labels.append(row["target_normal"])
    return np.array(images), np.array(labels)

train_images, train_labels = load_and_preprocess_images(train_df)
valid_images, valid_labels = load_and_preprocess_images(valid_df)

# Convert the data to TensorFlow Dataset
train_ds = Dataset.from_tensor_slices((train_images, train_labels))
valid_ds = Dataset.from_tensor_slices((valid_images, valid_labels))

# Shuffle the training dataset
train_ds = train_ds.shuffle(buffer_size=10000, reshuffle_each_iteration=True).batch(16)
valid_ds = valid_ds.batch(16)

# Define the model
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

# Define callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_loss')

# Train the model
history = model.fit(train_ds, epochs=50, validation_data=valid_ds, callbacks=[early_stopping])


In [ ]:
###### APPROACH 2 ######

df2 = pd.read_csv(r'C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\full_df.csv').rename(columns=lambda x: x.replace("-", "_").replace(" ", "_").lower())
df2 = df2.rename(columns = {'n': 'normal',
                        'd': 'diabetes',
                        'g': 'glaucoma',
                        'c': 'cataract',
                        'a': 'age_related_macular_degeneration',
                        'h': 'hypertension',
                        'm': 'pathological_myopia',
                        'o': 'other_diseases'})

df2 = pd.concat([df2, pd.get_dummies(df2["target"], prefix="target")], axis = 1)
df2 = df2.rename(columns = {'target_[1, 0, 0, 0, 0, 0, 0, 0]': 'target_normal',
                        'target_[0, 1, 0, 0, 0, 0, 0, 0]': 'target_diabetes',
                        'target_[0, 0, 1, 0, 0, 0, 0, 0]': 'target_glaucoma',
                        'target_[0, 0, 0, 1, 0, 0, 0, 0]': 'target_cataract',
                        'target_[0, 0, 0, 0, 1, 0, 0, 0]': 'target_age_related_macular_degeneration',
                        'target_[0, 0, 0, 0, 0, 1, 0, 0]': 'target_hypertension',
                        'target_[0, 0, 0, 0, 0, 0, 1, 0]': 'target_pathological_myopia',
                        'target_[0, 0, 0, 0, 0, 0, 0, 1]': 'target_other_diseases'})


df_normal_vs_rest = df2.copy()

df_normal_vs_rest = df_normal_vs_rest[["filename", "target_normal"]]

# Assuming df_normal_vs_rest is your dataframe
train_df, valid_df = train_test_split(df_normal_vs_rest, test_size=0.2, random_state=42)

img_size = 224
num_classes = 2
data_dir = r'C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images'

def process_path(file_path, label):
    # Load the raw data from the file as a string
    img = tf.io.read_file(data_dir + '/' + file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [img_size, img_size])
    img = img / 255.0  # normalize to [0,1] range
    label = tf.one_hot(label, depth=num_classes)
    return img, label

def prepare_dataset(df, batch_size=16):
    file_paths = df['filename']
    labels = df['target_normal']
    image_count = df.shape[0]
    ds = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    ds = ds.shuffle(buffer_size=image_count) # Buffer size should be the same as the total number of images to ensure a uniform shuffle.
    ds = ds.map(process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return ds



# Create train and validation datasets
train_ds = prepare_dataset(train_df)
valid_ds = prepare_dataset(valid_df)

# Define the model architecture
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

print("Define model: done ")

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

print("Compile model: done ")

# Define callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(patience=15, monitor='val_loss')

# Train the model
history = model.fit(train_ds, epochs=50, validation_data=valid_ds, callbacks=[early_stopping])

# Evaluate the model
# ...


In [ ]:
# Visualize one batch
for images, labels in train_ds.take(2):
    # Take the first image from the batch
    image = images[0]
    label = labels[0]

    plt.imshow(image)
    plt.title(f'Label: {label.numpy()}')
    plt.axis('off')
    plt.show()

In [ ]:
df_normal_vs_rest[df_normal_vs_rest["target_normal"] == 0].count()

In [ ]:
###### APPROACH 3 ######

df_normal_vs_rest = df2.copy()

df_normal_vs_rest = df_normal_vs_rest[["filename", "target_normal"]]

# Split df in train and test df

img_size = 224
data_dir = r'C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images'


# array_normal = np.asarray([(cv2.resize(cv2.imread(os.path.join(data_dir, elem))[...,::-1], (img_size, img_size)), 1) if (os.path.exists(os.path.join(data_dir, elem)) and df_normal_vs_rest[df_normal_vs_rest['filename'] == elem]['target_normal'].iloc[0] == 1) else (cv2.resize(cv2.imread(os.path.join(data_dir, elem))[...,::-1], (img_size, img_size)), 0) for elem in df_normal_vs_rest['filename']],dtype = object)
data_normal = [[cv2.resize(cv2.imread(os.path.join(data_dir, elem))[...,::-1], (img_size, img_size)), 1] if (os.path.exists(os.path.join(data_dir, elem)) and df_normal_vs_rest[df_normal_vs_rest['filename'] == elem]['target_normal'].iloc[0] == 1) else [cv2.resize(cv2.imread(os.path.join(data_dir, elem))[...,::-1], (img_size, img_size)), 0] for elem in df_normal_vs_rest['filename']]

# count_ones = np.count_nonzero(np.array(array_normal)[:, 1] == 1)
# count_zeros = np.count_nonzero(np.array(array_normal)[:, 1] == 0)

# print("Number of ones:", count_ones)
# print("Number of zeros:", count_zeros)

# shuffled_array = array_normal.copy()  # Create a copy of the array to preserve the original

random.shuffle(data_normal)  # Shuffle the array in-place

# # Extract images and labels from array_normal
# images = np.array([item[0]/255 for item in data_normal])
# labels = np.array([item[1] for item in data_normal])

# # Split the data into train and validation sets
# x_train, x_valid, y_train, y_valid = train_test_split(images, labels, test_size=0.2, random_state=42)

# # Convert the NumPy arrays to TensorFlow tensors
# x_train = tf.convert_to_tensor(x_train.astype('float32'))
# y_train = tf.convert_to_tensor(y_train.astype('float32'))
# x_valid = tf.convert_to_tensor(x_valid.astype('float32'))
# y_valid = tf.convert_to_tensor(y_valid.astype('float32'))

x_train = []
y_train = []
x_valid = []
y_valid = []

limit_train = round((len(data_normal)*(4/5)))
train_data = data_normal[:limit_train]
valid_data = data_normal[limit_train:]

for img, label in train_data:
    x_train.append(img/255)
    y_train.append(label)

for img, label in valid_data:
    x_valid.append(img/255)
    y_valid.append(label)
        

print(len(x_train))
print(len(y_train))


# Convert the data to NumPy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)
x_valid = np.array(x_valid)
y_valid = np.array(y_valid)

# Convert the NumPy arrays to TensorFlow tensors
x_train = tf.convert_to_tensor(x_train.astype('float32'))
y_train = tf.convert_to_tensor(y_train.astype('float32'))
x_valid = tf.convert_to_tensor(x_valid.astype('float32'))
y_valid = tf.convert_to_tensor(y_valid.astype('float32'))

from tensorflow.keras.utils import to_categorical
num_classes = 2  # Number of classes in your classification task
y_train = to_categorical(y_train, num_classes)
y_valid = to_categorical(y_valid, num_classes)

# Define the model architecture
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

print("Define model: done ")

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

print("Compile model: done ")

# Define callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss')

print("Training model...")
# Train the model
history = model.fit(x_train, y_train, epochs=50, 
                    validation_data=(x_valid, y_valid),
                    callbacks=[early_stopping])


# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_features, test_targets)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

In [ ]:
# # Split the data into train and validation sets
# x_train, x_valid, y_train, y_valid = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
##
#Second try
##
# img_size = 224
# data_dir = r'C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images'
# df_normal_vs_rest = ...  # Your dataframe containing image filenames and labels

# # Prepare the dataset
# data = []
# for filename, target in zip(df_normal_vs_rest['filename'], df_normal_vs_rest['target_normal']):
#     file_path = os.path.join(data_dir, filename)
#     if os.path.exists(file_path):
#         img = cv2.imread(file_path)[:,:, ::-1]  # Read and convert BGR to RGB
#         img = cv2.resize(img, (img_size, img_size))
#         data.append((img, target))
        
# array_normal = np.asarray([(cv2.resize(cv2.imread(os.path.join(data_dir, elem))[...,::-1], (img_size, img_size)), 1) if (os.path.exists(os.path.join(data_dir, elem)) and df_normal_vs_rest[df_normal_vs_rest['filename'] == elem]['target_normal'].iloc[0] == 1) else (cv2.resize(cv2.imread(os.path.join(data_dir, elem))[...,::-1], (img_size, img_size)), 0) for elem in df_normal_vs_rest['filename']],dtype = object)

# # Convert the list to numpy array
# array_normal = np.array(data, dtype=object)

# # Shuffle the array
# np.random.shuffle(array_normal)

# # Split the data into train and validation sets
# x_train, x_valid, y_train, y_valid = train_test_split(array_normal[:, 0], array_normal[:, 1], test_size=0.2, random_state=42)

# # Convert the NumPy arrays to TensorFlow tensors
# x_train = tf.convert_to_tensor(np.stack(x_train).astype('float32'))
# y_train = tf.convert_to_tensor(y_train.astype('float32'))
# x_valid = tf.convert_to_tensor(np.stack(x_valid).astype('float32'))
# y_valid = tf.convert_to_tensor(y_valid.astype('float32'))

# # Define the model architecture
# model = tf.keras.Sequential([
#     layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64, (3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Flatten(),
#     layers.Dense(64, activation='relu'),
#     layers.Dense(2, activation='softmax')
# ])

# # Compile the model
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])

# # Define callbacks
# early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss')

# # Train the model
# history = model.fit(x_train, y_train, epochs=50,
#                     validation_data=(x_valid, y_valid),
#                     callbacks=[early_stopping])

# # Evaluate the model
# test_loss, test_accuracy = model.evaluate(x_test, y_test)

# print("Test Loss:", test_loss)
# print("Test Accuracy:", test_accuracy)


In [ ]:


# # Extract images and labels from array_normal
# images = np.array([item[0]/255 for item in shuffled_array])
# labels = np.array([item[1] for item in shuffled_array])

# # Split the data into train and validation sets
# x_train, x_valid, y_train, y_valid = train_test_split(images, labels, test_size=0.2, random_state=42)

# # Convert the NumPy arrays to TensorFlow tensors
# x_train = tf.convert_to_tensor(x_train.astype('float32'))
# y_train = tf.convert_to_tensor(y_train.astype('float32'))
# x_valid = tf.convert_to_tensor(x_valid.astype('float32'))
# y_valid = tf.convert_to_tensor(y_valid.astype('float32'))

In [ ]:
from tensorflow.keras.utils import to_categorical

# Convert target data to one-hot encoded format
y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)

In [ ]:
# Define the model architecture
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

print("Define model: done ")

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

print("Compile model: done ")

# Define callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss')

print("Training model...")
# Train the model
history = model.fit(x_train, y_train, epochs=50, 
                    validation_data=(x_valid, y_valid),
                    callbacks=[early_stopping])


# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_features, test_targets)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


In [ ]:
pd.DataFrame(history.history).plot(figsize = (8,5))
plt.grid(True)
plt.gca()
plt.show()

In [ ]:
x_train[0][0][0]

In [ ]:
counts = df_normal_vs_rest["target_normal"].value_counts()

count_zeros = counts[0]
count_ones = counts[1]

print("Number of zeros:", count_zeros)
print("Number of ones:", count_ones)

In [ ]:
df_normal_vs_rest.shape

In [ ]:
data_array_normal[0:20, 1]

In [ ]:
count_ones = np.count_nonzero(np.array(data_array_normal)[:, 1] == 1)
count_zeros = np.count_nonzero(np.array(data_array_normal)[:, 1] == 0)


print("Number of ones:", count_ones)
print("Number of zeros:", count_zeros)

In [ ]:
(data_array_normal[:, 1] == 1).shape

### Problema abordado: Normal vs diabetes

In [ ]:
df_normal_diabetes = df2[((df2["target_normal"] == 0) & (df2['target_diabetes'] == 1)) | ((df2["target_normal"] == 1) & (df2['target_diabetes'] == 0))].copy()
df_normal_diabetes = df_normal_diabetes[['filename', 'target_normal','target_diabetes']]


# Create new column 'diabetes'
df_normal_diabetes['diabetes'] = df_normal_diabetes.apply(lambda x: 0 if x['target_normal'] == 1 else x['target_diabetes'], axis=1)

# # Drop original columns
df_normal_diabetes.drop(columns=['target_normal', 'target_diabetes'], inplace=True)

# Split df in train and test df

img_size = 224
data_dir = r'C:\Users\jdani\01_Development\01_Proyectos_Deep_Learning\archive\preprocessed_images'


# Filter the dataframe to get rows where diabetes is equal to 1
df_filtered_diabetes = df_normal_diabetes.loc[df_normal_diabetes['diabetes'] == 1]

# Use a list comprehension to create the data_array_diabetes list
data_array_diabetes = [[cv2.resize(cv2.imread(os.path.join(data_dir, elem))[...,::-1], (img_size, img_size)), 1] for elem in df_filtered_diabetes["filename"] if os.path.exists(os.path.join(data_dir, elem))]


# Filter the dataframe to get rows where normal is equal to 0
df_filtered_normal = df_normal_diabetes.loc[df_normal_diabetes['diabetes'] == 0]

# Use a list comprehension to create the data_array_normal list
data_array_normal = [[cv2.resize(cv2.imread(os.path.join(data_dir, elem))[...,::-1], (img_size, img_size)), 0] for elem in df_filtered_normal["filename"] if os.path.exists(os.path.join(data_dir, elem))]

    


In [ ]:
print(len(df_filtered_diabetes), " ",len(df_filtered_normal))

In [ ]:
# 1 index: numero de imagen [0:len(df)]
# 2 index: array de pixeles o clase [0:1]
# 3 index: cada pixel de cada imágen [0:224]. hemos definido este tamaño
array_diabetes_normal = np.random.permutation(np.concatenate((data_array_diabetes, data_array_normal)))

len(array_diabetes_normal)

In [ ]:
round((len(array_diabetes_normal)*(2/3)))

In [ ]:
a,b = array_diabetes_normal[4480]
print(np.size(a),b)

test =[]
test.append(a)
np.size(test[0][0][0])

In [ ]:
x_train = []
y_train = []
x_val = []
y_val = []

limit_train = round((len(array_diabetes_normal)*(2/3)))
i = 0
for img, label in array_diabetes_normal:
    if i >= limit_train:
        x_val.append(img)
        y_val.append(label)
        i+=1
    else:
        x_train.append(img)
        y_train.append(label)
        i+=1

# Normalize the data
x_train = np.array(x_train) / 255
x_val = np.array(x_val) / 255

# x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)

# x_val.reshape(-1, img_size, img_size, 1)
y_val = np.array(y_val)

In [ ]:
x_train[]

In [ ]:
(y_train == 1).sum()

In [ ]:
(y_train == 0).sum()

In [ ]:
len(y_train)

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

In [ ]:
model = Sequential()
model.add(Conv2D(32,3,padding="same", activation="relu", input_shape=(224,224,3)))
model.add(MaxPool2D())

model.add(Conv2D(32, 3, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Conv2D(64, 3, padding="same", activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(2, activation="softmax"))

model.summary()

In [ ]:
opt = Adam(learning_rate=0.000001)
model.compile(optimizer = opt , loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) , metrics = ['accuracy'])
# https://stackoverflow.com/questions/48285129/saving-best-model-in-keras

In [ ]:
datagen.flow(x_train, y_train, batch_size=32,
         subset='training',shuffle  = True)


In [ ]:
history = model.fit(datagen.flow(x_train, y_train, batch_size=32,
         subset='training'),epochs = 100 , validation_data = (x_val, y_val))

In [ ]:
pd.DataFrame(history.history).plot(figsize = (8,5))
plt.grid(True)
plt.gca()
plt.show()

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
plt.figure(figsize = (5,5))
plt.imshow(array_diabetes_normal[0][0])

In [ ]:
data_array_normal[0][1]

In [ ]:
np.size(df_filtered_diabetes)

In [ ]:
a

In [ ]:
len(range(0,len(df_normal_diabetes)))

In [ ]:
# sns.histplot(df_normal_diabetes["diabetes"])

##